In [1]:
import os
import re
import PIL
import sys
import json
import time
import timm
import math
import copy
import torch
import pickle
import logging
import fnmatch
import argparse
import itertools
import torchvision
import numpy as np
%matplotlib inline
import pandas as pd
# from apex import amp
import seaborn as sns
import albumentations
import torch.nn as nn
from PIL import Image
from pathlib import Path
from copy import deepcopy
import scikitplot as skplt
from sklearn import metrics
import torch.optim as optim
from timm import create_model
from datetime import datetime
from timm.data.loader import *
from torchvision import models
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data
# from cutmix.cutmix import CutMix
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
from torch.optim import lr_scheduler
#from pytorch_metric_learning import loss
import torch.utils.model_zoo as model_zoo
from timm.models.layers.activations import *
from timm.utils import accuracy, AverageMeter
%config InlineBackend.figure_format = 'retina'
# from apex.parallel import convert_syncbn_model
from timm.utils import ApexScaler, NativeScaler
# from cutmix.utils import CutMixCrossEntropyLoss
from timm.models.registry import register_model
from timm.models.vision_transformer import VisionTransformer

# import VIT_imagenet.VIT as v
from collections import OrderedDict, defaultdict
from torch.utils.tensorboard import SummaryWriter
from warmup_scheduler import GradualWarmupScheduler
from timm.models.resnet import resnet26d, resnet50d
from torchvision import transforms, models, datasets
from timm.models.helpers import build_model_with_cfg
from torch.utils.data.sampler import SubsetRandomSampler
from randaugment import RandAugment, ImageNetPolicy, Cutout
# from apex.parallel import DistributedDataParallel as ApexDDP
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.models.efficientnet import efficientnet_b0, efficientnet_b1, efficientnet_b2, efficientnet_b3
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, roc_curve, auc, roc_auc_score
#from timm.data import Dataset, DatasetTar, RealLabelsImagenet, create_loader, Mixup, FastCollateMixup, AugMixDataset

In [2]:
torch.backends.cudnn.benchmark = True
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
imagenet_dir = '/scratch/work/public/imagenet/train'

device = torch.device("cuda:0")

batch_size = 50
num_epochs = 500
# lr = 0.001
lr = 1e-4
beta = 1
step_size = 130
img_size = 224
test_size = 224


mean = [0.485] 
std = [0.229]
num_workers = 4
# Define your transforms for the training and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(test_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(test_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
}
print(device)

cuda:0


## ImageNet

In [4]:
train_split_size = 1000
val_split_size = 200

In [5]:
imagenet_train = datasets.ImageFolder(imagenet_dir, transform=data_transforms['train']
                                          )


#split into train and val samples 
train_split = torch.utils.data.random_split(imagenet_train, 
                                            [train_split_size, len(imagenet_train)-train_split_size])[0]

val_split = torch.utils.data.random_split(imagenet_train, 
                                            [len(imagenet_train)-val_split_size, val_split_size])[1]


# get train and val dataloaders
train = torch.utils.data.DataLoader(train_split, batch_size = batch_size, shuffle=True,
                                    num_workers=num_workers)

val = torch.utils.data.DataLoader(val_split, batch_size = batch_size, shuffle=True,
                                    num_workers=num_workers)

/ext3/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#### Check Train

In [ ]:
type(train)

In [ ]:
len(train)

In [ ]:
batch0 = next(iter(train))
batch0

In [ ]:
len(batch0)

In [ ]:
image0 = batch0[0][0]
label0 = batch0[1][0]
print('image:',image0)
print('label:',label0)
print('image size:',image0.size())

#### Check Val

In [ ]:
type(val)

In [ ]:
len(val)

In [ ]:
batch1 = next(iter(val))
batch1

In [ ]:
len(batch1)

In [ ]:
image1 = batch1[0][0]
label1 = batch1[1][0]
print('image:',image1)
print('label:',label1)
print('image size:',image1.size())

### CIFAR 10

from https://github.com/jeonggunlee/Vision-Transformer-Study/blob/main/Vision_Transformer_Cifar10.ipynb

In [6]:
patch = 4
batch_size = 50

In [7]:
train_split_size = 10000
val_split_size = 2000

In [8]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.CenterCrop(32),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

cifar_train = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
cifar_test = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

#split into train and val samples 
# cifar_train_split = torch.utils.data.random_split(cifar_train, 
#                                             [train_split_size, len(cifar_train)-train_split_size])[0]

# cifar_val_split = torch.utils.data.random_split(cifar_train, 
#                                             [len(cifar_train)-val_split_size, val_split_size])[1]

# train and val dataloaders
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=100, shuffle=False, num_workers=num_workers)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


#### CIFAR Check

In [ ]:
batch2 = next(iter(trainloader))
batch2

In [ ]:
len(batch2)

In [ ]:
batch2[0]

In [ ]:
batch2[0].size()

In [ ]:
batch2[1]

In [ ]:
batch2[1].size()

### CIFAR Model

In [9]:
cifar_model = VisionTransformer(
                                img_size = 32,
                                patch_size = patch,
                                num_classes = 10,
                                embed_dim = 512,
                                depth = 6,
                                num_heads = 8,
#                                 mlp_dim = 512,
#                                 dropout = 0.1,
#                                 emb_dropout = 0.1
                                )
cifar_model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 512, kernel_size=(4, 4), stride=(4, 4))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=512, out_features=1536, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=512, out_features=512, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((512,), eps

In [10]:
for param in cifar_model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cifar_model.parameters(), lr=lr,)
cifar_model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count = count_parameters(cifar_model)
print("The number of parameters of the model is:", count)

The number of parameters of the model is: 18979338


# Define Model

In [ ]:
model = VisionTransformer()
model

In [ ]:
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr,)
model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count = count_parameters(model)
print("The number of parameters of the model is:", count)

In [16]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=100):
    
    total_loss = 0.0
    total_correct = 0
    total_targets = 0
    best_acc = 0

    for e in range(num_epochs):

        epoch_loss = 0.0
        epoch_correct = 0
        epoch_targets = 0

        for i, data in enumerate(dataloader, 0):
            # extract inputs and labels
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero parameter gradients
            optimizer.zero_grad()

            # set model to training mode
            model.train()

            # forward pass
            outputs = model(inputs)

            # compute loss
            loss = criterion(outputs, labels)

            # backprop
            loss.backward()

            # optimizer step
            optimizer.step()

            # getting outputs in true/false 
            _, preds = outputs.max(1)
    #         _, preds = torch.max(outputs, 1)


            # ------------ EVALUATION -----------

            # compute number of correct predictions
            correct = torch.sum(preds == labels.data)

            # update correct counters
            epoch_correct += correct
            total_correct += correct

            # update loss counters
            epoch_loss += loss.item()
            total_loss += loss.item()

            # update target counters
            epoch_targets += labels.size(0)
            total_targets += labels.size(0)


            # calculate batch accuracy
            batch_acc = correct / labels.size(0) * 100


            # display every five batches
            if i % 5 == 0:
                print('Epoch: {0}/{1}, Batch: {2}/{3}, Loss: {4}, Acc: {5} %'.format(e+1,
                                                                              num_epochs,
                                                                              i,
                                                                              len(dataloader),
                                                                              loss.item(),
                                                                              batch_acc       
                                                                             ))

        # calculate epoch metrics
        epoch_loss = epoch_loss / len(dataloader)
        epoch_acc = epoch_correct / epoch_targets * 100
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc

        # calculate running accuracy
        running_acc = total_correct / total_targets * 100

        print('-'*20)
        print('Epoch {0} finished. Accuracy: {1} %     Loss: {2}'.format(e+1,
                                                                        epoch_acc,
                                                                        epoch_loss,
                                                                       ))
        print('Running Accuracy: {0} %'.format(running_acc))

        print('-'*20)


    total_acc = total_correct / total_targets *100

    print('Total accuracy: {0} %'.format(total_acc))
    print('Best accuracy: {0} %'.format(best_acc))
    
    return

In [17]:
train_model(cifar_model, dataloader=trainloader, criterion=criterion, optimizer=optimizer, num_epochs=50)

Epoch: 1/50, Batch: 0/1000, Loss: 2.38167142868042, Acc: 9.999999046325684 %
Epoch: 1/50, Batch: 5/1000, Loss: 2.0408551692962646, Acc: 29.999998092651367 %
Epoch: 1/50, Batch: 10/1000, Loss: 2.1622257232666016, Acc: 16.0 %
Epoch: 1/50, Batch: 15/1000, Loss: 2.0297794342041016, Acc: 28.0 %
Epoch: 1/50, Batch: 20/1000, Loss: 2.359875440597534, Acc: 12.0 %
Epoch: 1/50, Batch: 25/1000, Loss: 1.9956213235855103, Acc: 32.0 %
Epoch: 1/50, Batch: 30/1000, Loss: 1.8304450511932373, Acc: 28.0 %
Epoch: 1/50, Batch: 35/1000, Loss: 2.226020574569702, Acc: 22.0 %
Epoch: 1/50, Batch: 40/1000, Loss: 1.912062644958496, Acc: 34.0 %
Epoch: 1/50, Batch: 45/1000, Loss: 1.972313642501831, Acc: 38.0 %
Epoch: 1/50, Batch: 50/1000, Loss: 2.0667669773101807, Acc: 14.0 %
Epoch: 1/50, Batch: 55/1000, Loss: 1.994786262512207, Acc: 26.0 %
Epoch: 1/50, Batch: 60/1000, Loss: 1.8485417366027832, Acc: 26.0 %
Epoch: 1/50, Batch: 65/1000, Loss: 1.9282859563827515, Acc: 28.0 %
Epoch: 1/50, Batch: 70/1000, Loss: 2.0788586

Epoch: 1/50, Batch: 580/1000, Loss: 1.5608205795288086, Acc: 38.0 %
Epoch: 1/50, Batch: 585/1000, Loss: 1.6654746532440186, Acc: 42.0 %
Epoch: 1/50, Batch: 590/1000, Loss: 1.4299430847167969, Acc: 44.0 %
Epoch: 1/50, Batch: 595/1000, Loss: 1.5915818214416504, Acc: 39.999996185302734 %
Epoch: 1/50, Batch: 600/1000, Loss: 1.453416109085083, Acc: 48.0 %
Epoch: 1/50, Batch: 605/1000, Loss: 1.5297095775604248, Acc: 44.0 %
Epoch: 1/50, Batch: 610/1000, Loss: 1.5062918663024902, Acc: 44.0 %
Epoch: 1/50, Batch: 615/1000, Loss: 1.5098304748535156, Acc: 44.0 %
Epoch: 1/50, Batch: 620/1000, Loss: 1.5491982698440552, Acc: 44.0 %
Epoch: 1/50, Batch: 625/1000, Loss: 1.4652137756347656, Acc: 44.0 %
Epoch: 1/50, Batch: 630/1000, Loss: 1.6065843105316162, Acc: 36.0 %
Epoch: 1/50, Batch: 635/1000, Loss: 1.4814419746398926, Acc: 42.0 %
Epoch: 1/50, Batch: 640/1000, Loss: 1.4745488166809082, Acc: 39.999996185302734 %
Epoch: 1/50, Batch: 645/1000, Loss: 1.4776787757873535, Acc: 44.0 %
Epoch: 1/50, Batch: 6

Epoch: 2/50, Batch: 145/1000, Loss: 1.210758924484253, Acc: 50.0 %
Epoch: 2/50, Batch: 150/1000, Loss: 1.353770136833191, Acc: 53.999996185302734 %
Epoch: 2/50, Batch: 155/1000, Loss: 1.317724347114563, Acc: 52.0 %
Epoch: 2/50, Batch: 160/1000, Loss: 1.4018206596374512, Acc: 53.999996185302734 %
Epoch: 2/50, Batch: 165/1000, Loss: 1.221021294593811, Acc: 53.999996185302734 %
Epoch: 2/50, Batch: 170/1000, Loss: 1.5939985513687134, Acc: 44.0 %
Epoch: 2/50, Batch: 175/1000, Loss: 1.123025894165039, Acc: 64.0 %
Epoch: 2/50, Batch: 180/1000, Loss: 1.5149428844451904, Acc: 58.0 %
Epoch: 2/50, Batch: 185/1000, Loss: 1.4124244451522827, Acc: 45.999996185302734 %
Epoch: 2/50, Batch: 190/1000, Loss: 1.306954026222229, Acc: 48.0 %
Epoch: 2/50, Batch: 195/1000, Loss: 1.5523228645324707, Acc: 44.0 %
Epoch: 2/50, Batch: 200/1000, Loss: 1.3078423738479614, Acc: 44.0 %
Epoch: 2/50, Batch: 205/1000, Loss: 1.3624414205551147, Acc: 50.0 %
Epoch: 2/50, Batch: 210/1000, Loss: 1.074924111366272, Acc: 66.0 %

Epoch: 2/50, Batch: 725/1000, Loss: 1.1911115646362305, Acc: 52.0 %
Epoch: 2/50, Batch: 730/1000, Loss: 1.0469087362289429, Acc: 59.999996185302734 %
Epoch: 2/50, Batch: 735/1000, Loss: 0.978119969367981, Acc: 66.0 %
Epoch: 2/50, Batch: 740/1000, Loss: 1.2278151512145996, Acc: 50.0 %
Epoch: 2/50, Batch: 745/1000, Loss: 1.101670742034912, Acc: 64.0 %
Epoch: 2/50, Batch: 750/1000, Loss: 1.0698596239089966, Acc: 59.999996185302734 %
Epoch: 2/50, Batch: 755/1000, Loss: 1.1530261039733887, Acc: 56.0 %
Epoch: 2/50, Batch: 760/1000, Loss: 1.0493425130844116, Acc: 59.999996185302734 %
Epoch: 2/50, Batch: 765/1000, Loss: 1.6490627527236938, Acc: 39.999996185302734 %
Epoch: 2/50, Batch: 770/1000, Loss: 1.3681012392044067, Acc: 39.999996185302734 %
Epoch: 2/50, Batch: 775/1000, Loss: 1.2373732328414917, Acc: 58.0 %
Epoch: 2/50, Batch: 780/1000, Loss: 1.2249019145965576, Acc: 59.999996185302734 %
Epoch: 2/50, Batch: 785/1000, Loss: 1.4777233600616455, Acc: 52.0 %
Epoch: 2/50, Batch: 790/1000, Loss

Epoch: 3/50, Batch: 295/1000, Loss: 1.3189259767532349, Acc: 50.0 %
Epoch: 3/50, Batch: 300/1000, Loss: 1.1272779703140259, Acc: 59.999996185302734 %
Epoch: 3/50, Batch: 305/1000, Loss: 1.2139548063278198, Acc: 56.0 %
Epoch: 3/50, Batch: 310/1000, Loss: 1.097551941871643, Acc: 66.0 %
Epoch: 3/50, Batch: 315/1000, Loss: 1.0918844938278198, Acc: 56.0 %
Epoch: 3/50, Batch: 320/1000, Loss: 1.0403262376785278, Acc: 62.0 %
Epoch: 3/50, Batch: 325/1000, Loss: 1.0848932266235352, Acc: 58.0 %
Epoch: 3/50, Batch: 330/1000, Loss: 1.0415364503860474, Acc: 70.0 %
Epoch: 3/50, Batch: 335/1000, Loss: 1.2016067504882812, Acc: 66.0 %
Epoch: 3/50, Batch: 340/1000, Loss: 1.2549344301223755, Acc: 59.999996185302734 %
Epoch: 3/50, Batch: 345/1000, Loss: 1.106465458869934, Acc: 59.999996185302734 %
Epoch: 3/50, Batch: 350/1000, Loss: 1.1359137296676636, Acc: 53.999996185302734 %
Epoch: 3/50, Batch: 355/1000, Loss: 0.9764755368232727, Acc: 68.0 %
Epoch: 3/50, Batch: 360/1000, Loss: 0.8237084746360779, Acc: 7

...

Epoch: 48/50, Batch: 325/1000, Loss: 0.0127596165984869, Acc: 100.0 %
Epoch: 48/50, Batch: 330/1000, Loss: 0.02613397128880024, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 335/1000, Loss: 0.00888058915734291, Acc: 100.0 %
Epoch: 48/50, Batch: 340/1000, Loss: 0.004776401910930872, Acc: 100.0 %
Epoch: 48/50, Batch: 345/1000, Loss: 0.024663517251610756, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 350/1000, Loss: 0.014706495217978954, Acc: 100.0 %
Epoch: 48/50, Batch: 355/1000, Loss: 0.08166441321372986, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 360/1000, Loss: 0.05808155611157417, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 365/1000, Loss: 0.02635144256055355, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 370/1000, Loss: 0.025684256106615067, Acc: 100.0 %
Epoch: 48/50, Batch: 375/1000, Loss: 0.06411948800086975, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 380/1000, Loss: 0.021395675837993622, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 385/1000, Loss: 0.019112886860966682, A

Epoch: 48/50, Batch: 870/1000, Loss: 0.014062860049307346, Acc: 100.0 %
Epoch: 48/50, Batch: 875/1000, Loss: 0.0174721609801054, Acc: 100.0 %
Epoch: 48/50, Batch: 880/1000, Loss: 0.02652200497686863, Acc: 100.0 %
Epoch: 48/50, Batch: 885/1000, Loss: 0.2086002677679062, Acc: 94.0 %
Epoch: 48/50, Batch: 890/1000, Loss: 0.08939487487077713, Acc: 96.0 %
Epoch: 48/50, Batch: 895/1000, Loss: 0.08438971638679504, Acc: 96.0 %
Epoch: 48/50, Batch: 900/1000, Loss: 0.08498092740774155, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 905/1000, Loss: 0.023883288726210594, Acc: 100.0 %
Epoch: 48/50, Batch: 910/1000, Loss: 0.03643904626369476, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 915/1000, Loss: 0.10898511856794357, Acc: 96.0 %
Epoch: 48/50, Batch: 920/1000, Loss: 0.11926347017288208, Acc: 96.0 %
Epoch: 48/50, Batch: 925/1000, Loss: 0.1045374795794487, Acc: 97.99999237060547 %
Epoch: 48/50, Batch: 930/1000, Loss: 0.07937344908714294, Acc: 96.0 %
Epoch: 48/50, Batch: 935/1000, Loss: 0.090269289

Epoch: 49/50, Batch: 410/1000, Loss: 0.027068927884101868, Acc: 97.99999237060547 %
Epoch: 49/50, Batch: 415/1000, Loss: 0.01474593672901392, Acc: 100.0 %
Epoch: 49/50, Batch: 420/1000, Loss: 0.004687854554504156, Acc: 100.0 %
Epoch: 49/50, Batch: 425/1000, Loss: 0.01883753575384617, Acc: 97.99999237060547 %
Epoch: 49/50, Batch: 430/1000, Loss: 0.0037235510535538197, Acc: 100.0 %
Epoch: 49/50, Batch: 435/1000, Loss: 0.05536985769867897, Acc: 96.0 %
Epoch: 49/50, Batch: 440/1000, Loss: 0.029655437916517258, Acc: 97.99999237060547 %
Epoch: 49/50, Batch: 445/1000, Loss: 0.06534233689308167, Acc: 96.0 %
Epoch: 49/50, Batch: 450/1000, Loss: 0.04475220665335655, Acc: 97.99999237060547 %
Epoch: 49/50, Batch: 455/1000, Loss: 0.01308977510780096, Acc: 100.0 %
Epoch: 49/50, Batch: 460/1000, Loss: 0.024589620530605316, Acc: 100.0 %
Epoch: 49/50, Batch: 465/1000, Loss: 0.1554931104183197, Acc: 91.99999237060547 %
Epoch: 49/50, Batch: 470/1000, Loss: 0.1282132863998413, Acc: 94.0 %
Epoch: 49/50, Ba

Epoch: 49/50, Batch: 950/1000, Loss: 0.05906181409955025, Acc: 100.0 %
Epoch: 49/50, Batch: 955/1000, Loss: 0.01836373470723629, Acc: 100.0 %
Epoch: 49/50, Batch: 960/1000, Loss: 0.008104857988655567, Acc: 100.0 %
Epoch: 49/50, Batch: 965/1000, Loss: 0.016096556559205055, Acc: 100.0 %
Epoch: 49/50, Batch: 970/1000, Loss: 0.0725986436009407, Acc: 97.99999237060547 %
Epoch: 49/50, Batch: 975/1000, Loss: 0.10143372416496277, Acc: 94.0 %
Epoch: 49/50, Batch: 980/1000, Loss: 0.25315630435943604, Acc: 91.99999237060547 %
Epoch: 49/50, Batch: 985/1000, Loss: 0.12047625333070755, Acc: 97.99999237060547 %
Epoch: 49/50, Batch: 990/1000, Loss: 0.22819775342941284, Acc: 94.0 %
Epoch: 49/50, Batch: 995/1000, Loss: 0.04085654765367508, Acc: 97.99999237060547 %
--------------------
Epoch 49 finished. Accuracy: 98.32999420166016 %     Loss: 0.0471704363480676
Running Accuracy: 90.25293731689453 %
--------------------
Epoch: 50/50, Batch: 0/1000, Loss: 0.09522648900747299, Acc: 97.99999237060547 %
Epoc

Epoch: 50/50, Batch: 485/1000, Loss: 0.005737323313951492, Acc: 100.0 %
Epoch: 50/50, Batch: 490/1000, Loss: 0.07844196259975433, Acc: 97.99999237060547 %
Epoch: 50/50, Batch: 495/1000, Loss: 0.01813092641532421, Acc: 100.0 %
Epoch: 50/50, Batch: 500/1000, Loss: 0.007079361006617546, Acc: 100.0 %
Epoch: 50/50, Batch: 505/1000, Loss: 0.012261493131518364, Acc: 100.0 %
Epoch: 50/50, Batch: 510/1000, Loss: 0.10167275369167328, Acc: 97.99999237060547 %
Epoch: 50/50, Batch: 515/1000, Loss: 0.010919745080173016, Acc: 100.0 %
Epoch: 50/50, Batch: 520/1000, Loss: 0.11562059074640274, Acc: 97.99999237060547 %
Epoch: 50/50, Batch: 525/1000, Loss: 0.21819688379764557, Acc: 94.0 %
Epoch: 50/50, Batch: 530/1000, Loss: 0.021101927384734154, Acc: 100.0 %
Epoch: 50/50, Batch: 535/1000, Loss: 0.024209531024098396, Acc: 97.99999237060547 %
Epoch: 50/50, Batch: 540/1000, Loss: 0.09222500771284103, Acc: 91.99999237060547 %
Epoch: 50/50, Batch: 545/1000, Loss: 0.023382794111967087, Acc: 97.99999237060547 %

In [18]:
def test_model(model, dataloader, criterion):
    total_loss = 0.0
    total_correct = 0
    total_targets = 0
    best_acc = 0
    
    for i, data in enumerate(dataloader, 0):
        # extract inputs and labels
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # set model to eval mode
        model.eval()

        # forward pass
        outputs = model(inputs)

        # compute loss
        loss = criterion(outputs, labels)

        # getting outputs in true/false 
        _, preds = outputs.max(1)
    #         _, preds = torch.max(outputs, 1)


        # ------------ EVALUATION -----------

        # compute number of correct predictions
        correct = torch.sum(preds == labels.data)

        # update correct counter
        total_correct += correct

        # update loss counter
        total_loss += loss.item()

        # update target counter
        total_targets += labels.size(0)


        # calculate batch accuracy
        batch_acc = correct / labels.size(0) * 100


        # display every five batches
        if i % 5 == 0:
            print('Epoch: {0}/{1}, Batch: {2}/{3}, Loss: {4}, Acc: {5} %'.format('',
                                                                          '',
                                                                          i,
                                                                          len(dataloader),
                                                                          loss.item(),
                                                                          batch_acc       
                                                                         ))
        
        
    total_acc = total_correct / total_targets *100

    print('Total accuracy: {0} %'.format(total_acc))
    
    return

In [19]:
test_model(cifar_model, dataloader=testloader, criterion=criterion)

Epoch: /, Batch: 0/100, Loss: 1.299494743347168, Acc: 77.0 %
Epoch: /, Batch: 5/100, Loss: 1.5655828714370728, Acc: 64.0 %
Epoch: /, Batch: 10/100, Loss: 1.5907244682312012, Acc: 70.0 %
Epoch: /, Batch: 15/100, Loss: 1.8466229438781738, Acc: 61.0 %
Epoch: /, Batch: 20/100, Loss: 1.3449500799179077, Acc: 72.0 %
Epoch: /, Batch: 25/100, Loss: 2.248284101486206, Acc: 62.0 %
Epoch: /, Batch: 30/100, Loss: 1.6299623250961304, Acc: 68.0 %
Epoch: /, Batch: 35/100, Loss: 1.7828823328018188, Acc: 66.0 %
Epoch: /, Batch: 40/100, Loss: 1.7513331174850464, Acc: 71.0 %
Epoch: /, Batch: 45/100, Loss: 1.919031023979187, Acc: 58.999996185302734 %
Epoch: /, Batch: 50/100, Loss: 1.7421702146530151, Acc: 66.99999237060547 %
Epoch: /, Batch: 55/100, Loss: 1.6928858757019043, Acc: 69.0 %
Epoch: /, Batch: 60/100, Loss: 1.7863422632217407, Acc: 66.0 %
Epoch: /, Batch: 65/100, Loss: 1.5168423652648926, Acc: 69.0 %
Epoch: /, Batch: 70/100, Loss: 1.886906385421753, Acc: 66.99999237060547 %
Epoch: /, Batch: 75/1